### Data Loading

In [ ]:
import json
import pydeck
import pickle
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import os
import shapefile

In [ ]:
# load location index
with open("../data/akl_loc_idx.pkl", 'rb') as f:
    loc_idx = pickle.load(f) # datazone to point index
    idx_loc = {v:k for k, v in loc_idx.items()} # point index to datazone    
    print(f" -- loaded location index with dimension {len(loc_idx)}")

# load time index
with open("../data/akl_t_idx.pkl", 'rb') as f:
    t_idx = pickle.load(f)
    print(f" -- loaded time index with dimension {len(t_idx)}")
    
# load precomputed odt
with open("../data/akl_odt.npy", 'rb') as f:
    odt = np.load(f)
    print(f" -- loaded odt cube with dimensions {odt.shape}")

In [ ]:
# show odt time rane
times = list(t_idx.keys())
print(min(times), "-", max(times))

In [ ]:
# load polygon data
with open("../data/akl/akl_polygons_id.geojson", 'r') as f:
    polys = json.load(f)

In [ ]:
# load shapefile of points (data zone population centeroids)
sf_path = "../data/akl_points.shp"
sf = shapefile.Reader(sf_path)
records = sf.records()
coords = {}
for i, r in enumerate(records):    
    coords[r[0]] = sf.shape(i).points[0]
sf.close()

In [ ]:
# load IMD
imd = pd.read_csv("../data/akl_imd.csv", index_col="DZ2018")
imd.head()

In [ ]:
# clinics
# these are currently manuallys set to the nearest data zone location (population centeroid)
clinics = pd.read_csv("../data/akl_clinics.csv", index_col="DZ2018")
clinics.head()

### Some basic plotting with pydeck

Can use this to select a location ID

In [ ]:
# deckgl show polygons and location ids
view_state = pydeck.ViewState(
    longitude=174.7633,
    latitude=-36.8485,
    zoom=11,    
    max_zoom=16,
    pitch=0,
    bearing=0
)

# default view
geojson = pydeck.Layer(
    "GeoJsonLayer",
    polys, # needs to be wgs84
    opacity=0.2,
    stroked=True,
    line_width_min_pixels=1,
    filled=True,           
    pickable=True,
    auto_highlight=True,
    get_fill_color=[128, 128, 128],
    get_line_color=[255, 255, 255],    
)

r = pydeck.Deck(
    layers=[geojson], 
    initial_view_state=view_state, 
    map_style='mapbox://styles/mapbox/light-v9',   
    tooltip = {
        "text": "Location: {id}"
    }
)
#r.to_html("geojson_layer.html", iframe_width="100%")
r.show()

### One source to all destinations with one-way journey threshold

In [ ]:
loc = 7600522
lon, lat = coords[loc]
origin = loc_idx[loc] # get odt index from location id
dt = odt[origin, :, :] # get destination-time matrix for this origin
print(dt.shape)

In [ ]:
# view dt matrix
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(np.transpose(dt))
ax.set_xlabel("location index")
ax.set_ylabel("time index")

In [ ]:
# compute mean, std travel time
mean_tt = np.nanmean(dt, axis=1).reshape(-1, 1)
std_tt = np.nanstd(dt, axis=1).reshape(-1, 1)

In [ ]:
# create dataframe
ids = np.array(list(idx_loc.values())).reshape(-1, 1)
d = np.concatenate((ids, mean_tt, std_tt), axis=1)
df = pd.DataFrame(d, columns=["id", "mean_tt", "std_tt"])
df = df.astype({'id': 'int32'})
df = df.dropna()

# join with imd
df = df.join(imd, on="id")

In [ ]:
# threshold by mean_tt
threshold = 60 # minutes
df = df[df["mean_tt"] < threshold]

In [ ]:
# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        df,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="mean_tt",
        color_continuous_scale="Viridis",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        df,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="std_tt",
        color_continuous_scale="Viridis",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### All locations to one destinationwith one-way journey threshold

In [ ]:
print(clinics["name"])

In [ ]:
# get clinic location by name
clinic_name = "Rehab Plus"
clinic_loc = clinics.index[clinics["name"] == clinic_name].tolist()[0]
print(clinic_name, clinic_loc)

In [ ]:
loc = clinic_loc # or just set any location id
lon, lat = coords[loc]
destination = loc_idx[loc] # get odt index from location id
ot = odt[:, destination, :] # get origin-time matrix for this destination
print(ot.shape)

In [ ]:
# view ot matrix
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(np.transpose(ot))
ax.set_xlabel("location index")
ax.set_ylabel("time index")

In [ ]:
# compute mean, std travel time
mean_tt = np.nanmean(ot, axis=1).reshape(-1, 1)
std_tt = np.nanstd(ot, axis=1).reshape(-1, 1)

In [ ]:
# create dataframe
ids = np.array(list(idx_loc.values())).reshape(-1, 1)
d = np.concatenate((ids, mean_tt, std_tt), axis=1)
df = pd.DataFrame(d, columns=["id", "mean_tt", "std_tt"])
df = df.astype({'id': 'int32'})
df = df.dropna()

# join with imd
df = df.join(imd, on="id")

In [ ]:
# threshold by mean_tt
threshold = 60 # minutes
df = df[df["mean_tt"] < threshold]

In [ ]:
# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        df,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="mean_tt",
        color_continuous_scale="Viridis",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# plot - note geojson data needs to be wgs84
fig = px.choropleth_mapbox(
        df,
        geojson=polys, 
        featureidkey="id",
        locations="id",        
        center = {"lat": lat, "lon": lon},
        mapbox_style="carto-positron",
        color="std_tt",
        color_continuous_scale="Viridis",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()